# Notebook 3: Introduction to FastAPI
This notebook aims to familiarize and play with fast api, understand the different components, docs, routes, async vs wsgi, query vs path parameter, get/post/delete/put, fastapi swagger, json format response, rest api priciniples, pydantic type hinting

## ⚠️ This is not going to be the final API

## Learning Objectives
By the end of this notebook, you'll understand:
- FastAPI fundamentals and setup
- Creating routes with different HTTP methods (GET, POST, PUT, DELETE)
- Path parameters vs Query parameters
- Request/Response models with Pydantic
- Async capabilities
- Automatic API documentation (Swagger)
- REST API principles in practice

## 1. Setting Up FastAPI

First, let's install FastAPI and its dependencies. FastAPI uses Uvicorn as the ASGI server.

In [1]:
# Install FastAPI and Uvicorn (run this in terminal if needed)
# !pip install fastapi uvicorn[standard] pydantic

# Import required libraries
from fastapi import FastAPI, Query, Path, HTTPException
from pydantic import BaseModel
import uvicorn
from typing import Optional
import asyncio
import threading


## 2. Creating Your First FastAPI App

FastAPI is a modern, fast web framework for building APIs. Let's create a simple app:

In [2]:
# Create a FastAPI instance
app = FastAPI(
    title="FastAPI Learning API",
    description="A simple API to learn FastAPI concepts",
    version="1.0.0"
)

# Basic root endpoint
@app.get("/")
def read_root():
    """Welcome endpoint - returns a simple greeting"""
    return {"message": "Welcome to FastAPI!", "status": "success"}

# Run fastapi app in a separate thread beacuse of notebook environment
# Genrally, you would run `uvicorn main:app --reload` in terminal or "fastapi dev main.py"  
# where main.py contains the app and not in a notebook
def run():
    uvicorn.run(app, host="127.0.0.1", port=8000)

threading.Thread(target=run).start()

print("✅ FastAPI app created successfully!")
print("💡 Tip: Once we run the server, you can access:")
print("   - API: http://127.0.0.1:8000")
print("   - Swagger docs: http://127.0.0.1:8000/docs")
print("   - ReDoc: http://127.0.0.1:8000/redoc")

✅ FastAPI app created successfully!
💡 Tip: Once we run the server, you can access:
   - API: http://127.0.0.1:8000
   - Swagger docs: http://127.0.0.1:8000/docs
   - ReDoc: http://127.0.0.1:8000/redoc


## 3. Understanding HTTP Methods (GET, POST, PUT, DELETE)

REST APIs use different HTTP methods for different actions:

In [3]:
# Simple in-memory database for demonstration
items_db = {
    1: {"name": "Laptop", "price": 999.99, "description": "Gaming laptop"},
    2: {"name": "Mouse", "price": 29.99, "description": "Wireless mouse"}
}

# GET - Retrieve data
@app.get("/items")
def get_items():
    """Get all items"""
    return {"items": items_db, "count": len(items_db)}

@app.get("/items/{item_id}")
def get_item(item_id: int):
    """Get a specific item by ID"""
    if item_id not in items_db:
        raise HTTPException(status_code=404, detail="Item not found")
    return {"item": items_db[item_id], "id": item_id}

print("✅ GET endpoints created!")
print("📖 GET is used to retrieve/read data")

✅ GET endpoints created!
📖 GET is used to retrieve/read data


## 4. Pydantic Models for Request/Response

Pydantic provides data validation and type hints. Let's create models:

In [4]:
# Pydantic models for type validation and documentation
class Item(BaseModel):
    name: str
    price: float
    description: Optional[str] = None
    
class ItemResponse(BaseModel):
    id: int
    name: str
    price: float
    description: Optional[str] = None
    
class ItemUpdate(BaseModel):
    name: Optional[str] = None
    price: Optional[float] = None
    description: Optional[str] = None

# POST - Create new data
@app.post("/items", response_model=ItemResponse)
def create_item(item: Item):
    """Create a new item"""
    new_id = max(items_db.keys()) + 1 if items_db else 1
    items_db[new_id] = item.dict()
    return ItemResponse(id=new_id, **items_db[new_id])

print("✅ Pydantic models and POST endpoint created!")
print("📝 POST is used to create new resources")
print("🔍 Pydantic automatically validates input data and generates API docs")

✅ Pydantic models and POST endpoint created!
📝 POST is used to create new resources
🔍 Pydantic automatically validates input data and generates API docs


In [5]:
# PUT - Update existing data
@app.put("/items/{item_id}", response_model=ItemResponse)
def update_item(item_id: int, item_update: ItemUpdate):
    """Update an existing item"""
    if item_id not in items_db:
        raise HTTPException(status_code=404, detail="Item not found")
    
    # Update only provided fields
    stored_item = items_db[item_id]
    update_data = item_update.dict(exclude_unset=True)  # Only get fields that were set
    
    for field, value in update_data.items():
        stored_item[field] = value
    
    return ItemResponse(id=item_id, **stored_item)

# DELETE - Remove data
@app.delete("/items/{item_id}")
def delete_item(item_id: int):
    """Delete an item"""
    if item_id not in items_db:
        raise HTTPException(status_code=404, detail="Item not found")
    
    deleted_item = items_db.pop(item_id)
    return {"message": "Item deleted successfully", "deleted_item": deleted_item}

print("✅ PUT and DELETE endpoints created!")
print("🔄 PUT is used to update existing resources")
print("🗑️ DELETE is used to remove resources")

✅ PUT and DELETE endpoints created!
🔄 PUT is used to update existing resources
🗑️ DELETE is used to remove resources


## 5. Path Parameters vs Query Parameters

Understanding how to pass data to your API endpoints:

In [6]:
# Path Parameters - part of the URL path
@app.get("/users/{user_id}/items/{item_id}")
def get_user_item(
    user_id: int = Path(..., description="ID of the user", ge=1),
    item_id: int = Path(..., description="ID of the item", ge=1)
):
    """Get an item for a specific user (using path parameters)"""
    return {
        "user_id": user_id,
        "item_id": item_id,
        "message": f"Item {item_id} for user {user_id}"
    }

# Query Parameters - after ? in the URL
@app.get("/search")
def search_items(
    q: Optional[str] = Query(None, description="Search query"),
    min_price: Optional[float] = Query(None, description="Minimum price", ge=0),
    max_price: Optional[float] = Query(None, description="Maximum price", ge=0),
    limit: int = Query(10, description="Number of results", le=100)
):
    """Search items with query parameters"""
    
    # Filter items based on query parameters
    filtered_items = items_db.copy()
    
    if q:
        # Simple search in name and description
        filtered_items = {
            k: v for k, v in filtered_items.items() 
            if q.lower() in v["name"].lower() or (v.get("description") and q.lower() in v["description"].lower())
        }
    
    if min_price is not None:
        filtered_items = {k: v for k, v in filtered_items.items() if v["price"] >= min_price}
    
    if max_price is not None:
        filtered_items = {k: v for k, v in filtered_items.items() if v["price"] <= max_price}
    
    # Limit results
    limited_items = dict(list(filtered_items.items())[:limit])
    
    return {
        "query": q,
        "filters": {"min_price": min_price, "max_price": max_price},
        "limit": limit,
        "results": limited_items,
        "total_found": len(limited_items)
    }

print("✅ Path and Query parameters demonstrated!")
print("🛣️ Path parameters: /users/{user_id}/items/{item_id}")
print("❓ Query parameters: /search?q=laptop&min_price=100&limit=5")

✅ Path and Query parameters demonstrated!
🛣️ Path parameters: /users/{user_id}/items/{item_id}
❓ Query parameters: /search?q=laptop&min_price=100&limit=5


## 6. Async vs Synchronous (ASGI vs WSGI)

FastAPI supports both sync and async operations. Here's the difference:

In [7]:
# Synchronous endpoint (blocking)
@app.get("/sync-task")
def sync_heavy_task():
    """Synchronous task that blocks the thread"""
    import time
    time.sleep(2)  # Simulates heavy computation
    return {"message": "Sync task completed", "duration": "2 seconds"}

# Asynchronous endpoint (non-blocking)
@app.get("/async-task")
async def async_heavy_task():
    """Asynchronous task that doesn't block the thread"""
    await asyncio.sleep(2)  # Simulates async I/O operation
    return {"message": "Async task completed", "duration": "2 seconds"}

# Multiple async tasks
@app.get("/multiple-tasks")
async def multiple_async_tasks():
    """Run multiple async tasks concurrently"""
    
    async def fetch_data(task_id: int):
        await asyncio.sleep(1)  # Simulate API call
        return f"Data from task {task_id}"
    
    # Run tasks concurrently
    tasks = [fetch_data(i) for i in range(3)]
    results = await asyncio.gather(*tasks)
    
    return {
        "message": "Multiple async tasks completed",
        "results": results,
        "total_time": "~1 second (concurrent execution)"
    }

print("✅ Async and sync endpoints created!")
print("⚡ Async functions can handle more concurrent requests")
print("🔄 Use async for I/O operations (database, API calls, file operations)")
print("🔒 Use sync for CPU-intensive tasks or simple operations")

✅ Async and sync endpoints created!
⚡ Async functions can handle more concurrent requests
🔄 Use async for I/O operations (database, API calls, file operations)
🔒 Use sync for CPU-intensive tasks or simple operations


## 7. Running the FastAPI Server

Now let's run our FastAPI application and explore the automatic documentation:

In [8]:
# Function to run the server (uncomment to run)
def run_server():
    """Run the FastAPI server"""
    uvicorn.run(app, host="127.0.0.1", port=8000, log_level="info")

# For demonstration, let's show what endpoints we've created
def show_routes():
    """Display all available routes"""
    routes = []
    for route in app.routes:
        if hasattr(route, 'methods') and hasattr(route, 'path'):
            routes.append({
                "path": route.path,
                "methods": list(route.methods),
                "name": route.name
            })
    return routes

print("🚀 Server ready! Here are all our API endpoints:")
for route in show_routes():
    print(f"  {route['methods']} {route['path']}")

print("\n💡 To start the server, uncomment and run: run_server()")
print("📖 Then visit:")
print("   - Swagger UI: http://127.0.0.1:8000/docs")
print("   - ReDoc: http://127.0.0.1:8000/redoc")
print("   - OpenAPI JSON: http://127.0.0.1:8000/openapi.json")

🚀 Server ready! Here are all our API endpoints:
  ['HEAD', 'GET'] /openapi.json
  ['HEAD', 'GET'] /docs
  ['HEAD', 'GET'] /docs/oauth2-redirect
  ['HEAD', 'GET'] /redoc
  ['GET'] /
  ['GET'] /items
  ['GET'] /items/{item_id}
  ['POST'] /items
  ['PUT'] /items/{item_id}
  ['DELETE'] /items/{item_id}
  ['GET'] /users/{user_id}/items/{item_id}
  ['GET'] /search
  ['GET'] /sync-task
  ['GET'] /async-task
  ['GET'] /multiple-tasks

💡 To start the server, uncomment and run: run_server()
📖 Then visit:
   - Swagger UI: http://127.0.0.1:8000/docs
   - ReDoc: http://127.0.0.1:8000/redoc
   - OpenAPI JSON: http://127.0.0.1:8000/openapi.json


## 8. Testing Our API

Let's test some endpoints to see them in action:

In [9]:
# Test our endpoints directly (without running the server)
from fastapi.testclient import TestClient

# Create a test client
client = TestClient(app)

print("🧪 Testing API endpoints:")
print("=" * 50)

# Test GET root endpoint
response = client.get("/")
print(f"GET / → Status: {response.status_code}")
print(f"Response: {response.json()}")
print()

# Test GET items
response = client.get("/items")
print(f"GET /items → Status: {response.status_code}")
print(f"Response: {response.json()}")
print()

# Test GET specific item
response = client.get("/items/1")
print(f"GET /items/1 → Status: {response.status_code}")
print(f"Response: {response.json()}")
print()

# Test POST - create new item
new_item = {
    "name": "Keyboard",
    "price": 79.99,
    "description": "Mechanical keyboard"
}
response = client.post("/items", json=new_item)
print(f"POST /items → Status: {response.status_code}")
print(f"Response: {response.json()}")
print()

# Test search with query parameters
response = client.get("/search?q=laptop&min_price=500")
print(f"GET /search?q=laptop&min_price=500 → Status: {response.status_code}")
print(f"Response: {response.json()}")
print()

print("✅ All tests completed successfully!")

INFO:     Started server process [2712]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


🧪 Testing API endpoints:
GET / → Status: 200
Response: {'message': 'Welcome to FastAPI!', 'status': 'success'}

GET /items → Status: 200
Response: {'items': {'1': {'name': 'Laptop', 'price': 999.99, 'description': 'Gaming laptop'}, '2': {'name': 'Mouse', 'price': 29.99, 'description': 'Wireless mouse'}}, 'count': 2}

GET /items/1 → Status: 200
Response: {'item': {'name': 'Laptop', 'price': 999.99, 'description': 'Gaming laptop'}, 'id': 1}

POST /items → Status: 200
Response: {'id': 3, 'name': 'Keyboard', 'price': 79.99, 'description': 'Mechanical keyboard'}

GET /search?q=laptop&min_price=500 → Status: 200
Response: {'query': 'laptop', 'filters': {'min_price': 500.0, 'max_price': None}, 'limit': 10, 'results': {'1': {'name': 'Laptop', 'price': 999.99, 'description': 'Gaming laptop'}}, 'total_found': 1}

✅ All tests completed successfully!


/var/folders/f5/kfsqzthj3pz6z0drx5433d5h0000gn/T/ipykernel_2712/857211421.py:23: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  items_db[new_id] = item.dict()


## 9. REST API Principles Summary

Let's review the key REST principles we've implemented:

### REST Principles:
1. **Resources**: Everything is a resource (items, users)
2. **HTTP Methods**: Use appropriate methods for actions
   - GET: Retrieve data
   - POST: Create new resources
   - PUT: Update existing resources
   - DELETE: Remove resources
3. **Stateless**: Each request contains all needed information
4. **Uniform Interface**: Consistent URL patterns and response formats
5. **JSON Format**: Standard data exchange format

### FastAPI Key Features:
1. **Automatic Documentation**: Swagger UI and ReDoc
2. **Type Hints**: Python type annotations for validation
3. **Pydantic Models**: Data validation and serialization
4. **Async Support**: High performance with async/await
5. **Path & Query Parameters**: Flexible data passing
6. **Error Handling**: HTTP exceptions for error responses

## 10. Next Steps & Practice Exercises

### Try These Exercises:
1. **Add a new endpoint**: Create `/health` endpoint that returns API health status
2. **Extend the Item model**: Add fields like `category`, `in_stock`, `created_date`
3. **Add validation**: Use Pydantic validators for price (must be positive) and name (min length)
4. **Create user endpoints**: Implement CRUD operations for users
5. **Add filtering**: Extend search to filter by category or stock status
6. **Error handling**: Add custom exception handlers
7. **Middleware**: Add logging middleware to track requests

### Production Considerations:
- **Database**: Replace in-memory storage with real database (PostgreSQL, MongoDB)
- **Authentication**: Add JWT tokens or OAuth2
- **Rate Limiting**: Prevent API abuse
- **CORS**: Configure for frontend applications
- **Environment Variables**: Use for configuration
- **Logging**: Comprehensive logging strategy
- **Testing**: Unit and integration tests
- **Deployment**: Docker, AWS, Heroku, etc.

### Useful Resources:
- [FastAPI Documentation](https://fastapi.tiangolo.com/)
- [Pydantic Documentation](https://pydantic-docs.helpmanual.io/)
- [Uvicorn Documentation](https://www.uvicorn.org/)
- [REST API Best Practices](https://restfulapi.net/)